## Load in the distinct queries and merge into a single dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
csv_names = ["images.csv", "measurementOrFact.csv", "occurrences.csv"]
query_dirs = [x for x in os.listdir() if (x.startswith("query") and os.path.isdir(x))]
merged_dir = "continental_US_CCH2_temp"
os.makedirs(merged_dir)
print("Query Directories:", query_dirs)

Query Directories: ['query2', 'query1', 'query4', 'query3']


In [3]:
for csv_filename in csv_names:
    merged_dataframe = pd.concat([pd.read_csv(q_dir + "/" + csv_filename) for q_dir in query_dirs])
    merged_dataframe.drop_duplicates(inplace=True)
    merged_dataframe.to_csv(merged_dir + "/" + csv_filename, index=False)

/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (3,26,28,42,43,44,50,56,65,67,73,75) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=as

In [4]:
cch2 = pd.read_csv(merged_dir + "/occurrences.csv")
phenology = pd.read_csv(merged_dir + "/measurementOrFact.csv")
len(cch2), len(phenology)

/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,26,28,29,42,43,44,50,56,65,67,73,75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/jackson/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1554807, 1679766)

## How to determine if a sample is flowering / fruiting?

In [5]:
ids_with_open_flower = phenology.loc[((phenology["measurementType"] == "Open Flower") &
                                      (phenology["measurementValue"] == "present")),
                                     ["coreid"]]
ids_with_open_flower["withOpenFlower"] = True
ids_with_open_flower.set_index("coreid", inplace=True)

ids_with_fruit = phenology.loc[((phenology["measurementType"] == "Fruit") &
                                (phenology["measurementValue"] == "present")),
                               ["coreid"]]
ids_with_fruit["withFruit"] = True
ids_with_fruit.set_index("coreid", inplace=True)

ids_reproductive = phenology.loc[((phenology["measurementType"] == "Angiosperm Phenological Traits") &
                                      (phenology["measurementValue"] == "reproductive")),
                                     ["coreid"]]
ids_not_reproductive = phenology.loc[((phenology["measurementType"] == "Angiosperm Phenological Traits") &
                                      (phenology["measurementValue"] == "sterile")),
                                     ["coreid"]]
ids_reproductive["reproductive"] = True
ids_not_reproductive["reproductive"] = False
ids_reproductive.set_index("coreid", inplace=True)
ids_not_reproductive.set_index("coreid", inplace=True)
ids_reproductive = pd.concat([ids_reproductive,ids_not_reproductive])

In [6]:
print("CCH2 shape before joining", cch2.shape)
cch2 = cch2.join(ids_with_open_flower, on="id")
cch2 = cch2.join(ids_with_fruit, on="id")
cch2 = cch2.join(ids_reproductive, on="id")
print("CCH2 shape after joining", cch2.shape)

CCH2 shape before joining (1554807, 84)
CCH2 shape after joining (1554807, 87)


In [7]:
indices_to_na = cch2.loc[(cch2["lifeStage"].isna()
                          & cch2["reproductiveCondition"].isna()
                          & cch2["withOpenFlower"].isna()
                          & cch2["withFruit"].isna())].index

cch2["reproductiveCondition"] = cch2["reproductiveCondition"].str.lower()

flowering_terms = [str(x) for x in cch2["reproductiveCondition"].unique() if "flower" in str(x)]

fruiting_terms = [str(x) for x in cch2["reproductiveCondition"].unique() if "fruit" in str(x)]

flowering_terms += ['ffl', 'ffr']
fruiting_terms += ['ffl', 'ffr', 'fr']

cch2["flowering"] = (cch2["lifeStage"].str.lower().isin(flowering_terms)
                     | cch2["reproductiveCondition"].isin(flowering_terms)
                     | cch2["withOpenFlower"])

cch2["fruiting"] = (cch2["lifeStage"].str.lower().isin(fruiting_terms)
                    | cch2["reproductiveCondition"].isin(fruiting_terms)
                    | cch2["withFruit"])

cch2.loc[indices_to_na, ["flowering", "fruiting"]] = np.nan

## Group all variants and subspecies together

In [8]:
cch2["scientificName"] = cch2["scientificName"].str.split(" var. ").str[0].str.split(" subsp. ").str[0]

guttata_sensu_lato_species = [
    "mimulus guttatus",
    "erythranthe guttata",
    "erythranthe micorphylla",
    "e. grandis",
    "e. lagunensis",
    "e. unimaculata",
    "e. thermalis",
    "e. arenicola",
    "e. marmorata",
    "e. pardali",
    "e. pardalis",
    "e. arvensis",
    "e. brachystylis",
    "e. charlestonensis",
    "e. cordata"
]

for sp in guttata_sensu_lato_species:
    if sp.startswith("e. "):
        guttata_sensu_lato_species.append("erythranthe" + sp[2:])
        
print(guttata_sensu_lato_species[-1])

erythranthe cordata


In [9]:
in_group = cch2["scientificName"].str.lower().isin(guttata_sensu_lato_species)
print("Number in guttata sensu lato:", in_group.sum())
cch2.loc[in_group, "scientificName"] = "Guttata sensu lato"

Number in guttata sensu lato: 2318


In [10]:
species_of_interest = [
    "Cuscuta campestris",
    "Trifolium pratense",
    "Ricinus communis",
    "Brassica rapa",
    "Raphanus sativus",
    "Nicotiana attenuata",
    "Helianthus annuus",
    "Guttata sensu lato"
]

for sp in species_of_interest:
    in_group = cch2.loc[(cch2["scientificName"].str.lower() == sp.lower())]
    print(sp, len(in_group))

Cuscuta campestris 180
Trifolium pratense 222
Ricinus communis 354
Brassica rapa 479
Raphanus sativus 769
Nicotiana attenuata 490
Helianthus annuus 563
Guttata sensu lato 2318


## Drop rows without "Year" and impute missing "startDayOfYear"

In [11]:
prev_len = len(cch2)
cch2.dropna(subset=["year"], inplace=True)
print("Dropped {} rows with NA year.".format(prev_len-len(cch2)))

cch2["year"] = cch2["year"].astype(int)

prev_len = len(cch2)
cch2.drop(cch2.loc[cch2["year"]<1895].index, axis=0, inplace=True)
print("Dropped {} rows with year<1895.".format(prev_len-len(cch2)))

prev_len = len(cch2)
cch2.drop(cch2.loc[(cch2["startDayOfYear"].isna() &
                    (cch2["month"].isna() |
                     (cch2["month"] < 1.0) |
                     (cch2["month"] > 12.0)))].index, axis=0, inplace=True)
print("Dropped {} rows with no month or startDayOfYear.".format(prev_len-len(cch2)))

Dropped 8676 rows with NA year.
Dropped 25537 rows with year<1895.
Dropped 7061 rows with no month or startDayOfYear.


In [12]:
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
estimate_from_month = [sum(month_days[:i])+(month_days[i]/2.0) for i in range(len(month_days))]
print(estimate_from_month)

[15.5, 45.0, 74.5, 105.0, 135.5, 166.0, 196.5, 227.5, 258.0, 288.5, 319.0, 349.5]


In [13]:
missing_indices = cch2.loc[cch2["startDayOfYear"].isna()].index
print("{} rows missing startDayOfYear".format(len(missing_indices)))
cch2.loc[missing_indices, "startDayOfYear"] = cch2.loc[missing_indices, "month"].apply(lambda x: estimate_from_month[int(x)-1])
print("After imputation, {} rows missing startDayOfYear".format(cch2["startDayOfYear"].isna().sum()))

63156 rows missing startDayOfYear
After imputation, 0 rows missing startDayOfYear


## Save the full CCH2 Dataset

In [14]:
cch2.to_csv(merged_dir + "/full_dataset.csv", index=False)